In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 51.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=1562ae8a792f31c17e42abbc8e5a8028f5ba4c59770ece1131c2c652270b9d3d
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [46]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import (DecisionTreeClassifier, GBTClassifier, RandomForestClassifier)
from pyspark.ml import pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [3]:
spark = SparkSession.builder.appName('cl').getOrCreate()

In [4]:
df = spark.read.csv('College.csv', header=True, inferSchema=True)

In [5]:
df.show()

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30|     12.2|         16| 10527|       56|
|      Adrian College|    Yes|1428|  1097|   336|       22|       50|       1036|         99|  

In [6]:
df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [9]:
df.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [11]:
df.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [12]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'], outputCol='features')

In [13]:
output = assembler.transform(df)

In [15]:
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndexer')

In [16]:
output_fixed = indexer.fit(output).transform(output)

In [17]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndexer: double (nullable = false)



In [18]:
final_df = output_fixed.select('features', 'PrivateIndexer')

In [21]:
final_df.show(5)

+--------------------+--------------+
|            features|PrivateIndexer|
+--------------------+--------------+
|[1660.0,1232.0,72...|           0.0|
|[2186.0,1924.0,51...|           0.0|
|[1428.0,1097.0,33...|           0.0|
|[417.0,349.0,137....|           0.0|
|[193.0,146.0,55.0...|           0.0|
+--------------------+--------------+
only showing top 5 rows



In [22]:
train_df, test_df = final_df.randomSplit([0.7, 0.3])

In [37]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndexer',featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndexer',featuresCol='features')
rfc = RandomForestClassifier(numTrees=150,labelCol='PrivateIndexer',featuresCol='features')

In [38]:
dtc_model = dtc.fit(train_df)
gbt_model = gbt.fit(train_df)
rfc_model = rfc.fit(train_df)

In [39]:
dtc_preds = dtc_model.transform(test_df)
gbt_preds = gbt_model.transform(test_df)
rfc_preds = rfc_model.transform(test_df)

In [40]:
binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndexer')

In [41]:
print(f'DTC: {binary_eval.evaluate(dtc_preds)}')

DTC: 0.9287671232876712


In [42]:
print(f'RFC: {binary_eval.evaluate(rfc_preds)}')

RFC: 0.9622146118721459


In [43]:
gbt_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndexer: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [44]:
binary_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndexer', rawPredictionCol='prediction')

In [45]:
print(f'GBT: {binary_eval2.evaluate(gbt_preds)}')

GBT: 0.8922374429223744


In [48]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndexer',metricName='accuracy')

In [49]:
print(f'RFC: {acc_eval.evaluate(rfc_preds)}')

RFC: 0.9174757281553398
